# 异常

## **Ruby**中的异常

写代码的时候，出现错误必不可免，即使代码没有问题，也可能遇到别的问题。异常提供了一种比较好的方式来提示错误。


### **Ruby**异常层次结构

```
Exception
 NoMemoryError
 ScriptError
   LoadError
   NotImplementedError
   SyntaxError
 SignalException
   Interrupt
 StandardError
   ArgumentError
   IOError
     EOFError
   IndexError
   LocalJumpError
   NameError
     NoMethodError
   RangeError
     FloatDomainError
   RegexpError
   RuntimeError
   SecurityError
   SystemCallError
   SystemStackError
   ThreadError
   TypeError
   ZeroDivisionError
 SystemExit
 fatal
```

### 异常处理代码

```
begin
    <statements>
rescue <args> [do]
    <statements>
else
    <statements>
ensure
    <statements>
end
```

`[do]` 可以使用 `do`, `\n` 及 `;` 代替，`rescue` 块可以出现多次，`else` 及 `ensure`可选。`rescue` 类似于其他语言中的 `catch`，`ensure` 类似于其他语言中的 `finally`。

#### 代码结构说明
- 执行 `begin` 块中的代码，
- 如果出现错误，则会寻找 `rescue` 中对应的异常块，如果找到，就执行对应的块，没有则抛出这个异常，
- 如果没有出错，则执行 `else` 块中的代码，
- `ensure` 块中的代码，无论前面是否有执行错误，都会被执行。

先来看一个例子：

In [13]:
begin
  x = 100 / 0
rescue  
  x = 0
end

x

0

## 异常处理

### 捕捉所有异常

```ruby
begin
  do_something_dangerous
rescue Exception
  do_something_when_error
end
```

或者

```ruby
begin
  do_something_really_dangerous
rescue Exception => e
  fix_unknown_error(e)
end
```

完整版本

```ruby
begin
  do_something_dangerous
rescue SomeError => e
  fix_some_error(e)
rescue SomeOtherError => e
  fix_some_other_error(e)
rescue Exception => e
  fix_unknown_error(e)
else
  something_dangerous_finished
ensure
  whether_or_not_an_error_was_raised
end
```

看个例子：

In [36]:
begin
  require 'nonexist'
rescue Exception => e
  puts e.class, e.message
else
  puts "code execute successful"
ensure
  puts "ensure code is executed"
end

LoadError
cannot load such file -- nonexist
ensure code is executed


### 处理标准异常

```ruby
begin
  do_something_dangerous
rescue
  do_something_when_error
end
```

或者

```ruby
begin
  do_something_dangerous
rescue => e
  fix_unknown_error(e)
end
```

如果不指定异常类型，默认可以处理 `StandardError` 下的所有子类。

In [35]:
begin
  1 / 0
rescue => e
  puts e.class, e.message
end

ZeroDivisionError
divided by 0


In [41]:
begin
  require 'nonexists'
rescue
  puts "just can handle standard error"
end

LoadError: cannot load such file -- nonexists

## 捕捉具体的错误类型

In [39]:
begin
  100 / 0
rescue ArgumentError, ZeroDivisionError => e
  puts "standard error:", e.message  
rescue LoadError => e
  puts "load error:", e.message  
end

standard error:
divided by 0


In [40]:
begin
  require 'nonexist'
rescue ArgumentError, ZeroDivisionError => e
  puts "standard error:", e.message  
rescue LoadError => e
  puts "load error:", e.message  
end

load error:
cannot load such file -- nonexist


## 获取异常的具体信息

可以使用 `Exception` 的方法 `message`, `backtrace` 获取更多的异常信息

In [49]:
begin
  1 / 0
rescue => e
  puts "MESSAGE: #{e.message}"
  puts "BACKTRACE: \n #{e.backtrace.join("\n")}"
end

MESSAGE: divided by 0
BACKTRACE: 
 <main>:1:in `/'
<main>:1:in `<main>'
/usr/local/rvm/gems/ruby-2.4.1/bundler/gems/iruby-a2d7f8bcd676/lib/iruby/backend.rb:72:in `eval'
/usr/local/rvm/gems/ruby-2.4.1/bundler/gems/iruby-a2d7f8bcd676/lib/iruby/backend.rb:72:in `block in eval'
/usr/local/rvm/gems/ruby-2.4.1/bundler/gems/iruby-a2d7f8bcd676/lib/iruby/backend.rb:55:in `eval_with_magic'
/usr/local/rvm/gems/ruby-2.4.1/bundler/gems/iruby-a2d7f8bcd676/lib/iruby/backend.rb:71:in `eval'
/usr/local/rvm/gems/ruby-2.4.1/bundler/gems/iruby-a2d7f8bcd676/lib/iruby/backend.rb:13:in `eval'
/usr/local/rvm/gems/ruby-2.4.1/bundler/gems/iruby-a2d7f8bcd676/lib/iruby/kernel.rb:87:in `execute_request'
/usr/local/rvm/gems/ruby-2.4.1/bundler/gems/iruby-a2d7f8bcd676/lib/iruby/kernel.rb:47:in `dispatch'
/usr/local/rvm/gems/ruby-2.4.1/bundler/gems/iruby-a2d7f8bcd676/lib/iruby/kernel.rb:37:in `run'
/usr/local/rvm/gems/ruby-2.4.1/bundler/gems/iruby-a2d7f8bcd676/lib/iruby/command.rb:70:in `run_kernel'
/usr/local/rvm/gem

## 抛出异常

使用 `raise` 可以快速抛出异常：

In [58]:
begin
  raise ArgumentError, "Here is a exception"
rescue => e
  puts e.class, e.message
end

ArgumentError
Here is a exception


也可以不指定具体异常类型，默认异常为 `RuntimeError`

In [59]:
begin
  raise "Here is a exception"
rescue => e
  puts e.class, e.message
end

RuntimeError
Here is a exception


自定义异常

In [60]:
class NoJuice < StandardError
end

def bang(power)
  raise NoJuice, "need at least 1.21 jigawatts, but only had #{power}" unless power >= 1.21
end

begin
  bang(1)
rescue => e
    puts e.class, e.message
end

NoJuice
need at least 1.21 jigawatts, but only had 1


## 其他

### `else`

如果 `begin` 代码块执行过程中，没有任何错误，则执行 `else` 代码块中的代码。 

In [61]:
begin
  puts "Here is no error"
rescue
  puts "never executed"
else
  puts "It should be executed"
end

Here is no error
It should be executed


### ensure

`ensure` 代码块中的代码，无论是 `begin` 代码块中的代码是否执行错误，甚至是 `else`, `ensure` 代码块中的代码是否执行错误，都会被执行。

In [62]:
begin
  puts "Here is no error"
rescue
  puts "never executed"
else
  puts "It should be executed"
ensure
  puts "finally executed"
end

Here is no error
It should be executed
finally executed


In [63]:
begin
  raise 'begin raise error'
rescue
  raise "rescue is error"
else
  puts "never executed"
ensure
  puts "finally executed"
end

finally executed


RuntimeError: rescue is error